# Data Prep

In [1]:
import pandas as pd
import numpy as np
import wandb

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import f1_score, matthews_corrcoef

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import psutil
import time
import logging
from datetime import datetime
import random
import os
import shutil
import itertools

import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.search.optuna import OptunaSearch
from ray.tune.tuner import Tuner, TuneConfig
from ray.train import RunConfig
from ray.tune import Trainable
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb

import helper

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Check if MPS (Apple Silicon) or CUDA (Nvidia) GPU is available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

2024-08-15 10:43:10,823	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Using device: mps


## Existing Data Files Check 

In [2]:
def clear_folder(folder_path):
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if item == '.gitignore':
            continue
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

clear_folder("ray_results")
clear_folder("outputs")
clear_folder("results")

## Simple FNN

In [3]:
class Classic_FFN(nn.Module):
    def __init__(self, num_classes, hidden_layers=[30, 15], dropout_rate=0.5, activation='relu'):
        super(Classic_FFN, self).__init__()

        self.activation = activation

        self.shared_layers = nn.ModuleList()

        input_dim = 30

        for hidden_layer in hidden_layers:
            self.shared_layers.append(nn.Linear(input_dim, hidden_layer))
            self.shared_layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_layer

        self.output_layer = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        for layer in self.shared_layers:
            if isinstance(layer, nn.Linear):
                x = layer(x)
                x = self.apply_activation(x)
            elif isinstance(layer, nn.Dropout):
                x = layer(x)
            
        x = self.output_layer(x)
        return x

    def apply_activation(self, x):
        if self.activation == 'relu':
            return F.relu(x)
        elif self.activation == 'tanh':
            return torch.tanh(x)
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)
        else:
            raise ValueError(f'Invalid activation: {self.activation}')

In [4]:
def create_model(num_classes, hidden_layers=[30,15], dropout_rate=0.5, activation='relu'):
    model = Classic_FFN(num_classes=num_classes, hidden_layers=hidden_layers, dropout_rate=dropout_rate, activation=activation)
    model.to(device)  # Move the model to the selected device
    return model

# def train(model, X_train, y_train, num_epochs, batch_size, learning_rate):
#     # Convert data to PyTorch tensors and move to the selected device
#     X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
#     y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long).to(device)  # Convert one-hot to class indices

#     # Create a DataLoader for the training data
#     train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#     train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
#     # Define loss function and optimizer
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
#     # Training loop
#     for epoch in range(num_epochs):
#         model.train()  # Set the model to training mode
#         for X_batch, y_batch in train_loader:
#             optimizer.zero_grad()  # Clear the gradients
#             outputs = model(X_batch)  # Forward pass
#             loss = criterion(outputs, y_batch)  # Calculate loss
#             loss.backward()  # Backward pass
#             optimizer.step()  # Update weights
    
#     return model

def train(model, X_train, y_train, num_epochs, batch_size, learning_rate):
    # Convert data to PyTorch tensors and move to the selected device
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long).to(device)  # Convert one-hot to class indices

    # Create a DataLoader for the training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Custom run name using trial ID and other config parameters
    run_name = f"trial_{session.get_trial_id()}"
    
    # Initialize WandB with custom run name
    wandb.init(project="Classic FFN PyTorch - Train Loss Logging", reinit=True, name=run_name)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Clear the gradients
            outputs = model(X_batch)  # Forward pass
            loss = criterion(outputs, y_batch)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            
            # If dataset is A12 then we take running loss += x * loss.item()
            running_loss += loss.item()

        # Calculate average loss over the epoch
        avg_loss = running_loss / len(train_loader)
        
        # Log the loss and epoch to WandB
        wandb.log({"loss": avg_loss, "epoch": epoch})
    
    return model


def evaluate(model, X_test, y_test):
    # Convert data to PyTorch tensors and move to the selected device
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(X_test_tensor)  # Forward pass
        y_pred_classes = torch.argmax(outputs, dim=1).cpu().numpy()  # Get predicted classes and move to CPU
        y_test_classes = torch.argmax(y_test_tensor, dim=1).cpu().numpy()  # Get true classes and move to CPU
        
        # Calculate accuracy
        accuracy = (y_pred_classes == y_test_classes).mean()
        
        # Calculate F1 scores and MCC
        f1_macro = f1_score(y_test_classes, y_pred_classes, average='macro')
        f1_micro = f1_score(y_test_classes, y_pred_classes, average='micro')
        mcc = matthews_corrcoef(y_test_classes, y_pred_classes)
    
    return accuracy, f1_macro, f1_micro, mcc

In [5]:
def evaluate_model_on_dataset_one_split(config, split, split_index):
    test_set = split[split_index]
    indices = [0, 1, 2, 3, 4]
    indices.remove(split_index)
    train_splits = [split[i] for i in indices]
    train_set = pd.concat(train_splits, axis=0)

    classes = train_set['track'].unique()
    num_classes = len(classes)
    #print("number of classes: " + str(num_classes))

    one_hot_columns = train_set['track'].unique()
    one_hot = pd.get_dummies(train_set['track'])
    train_set = train_set.drop('track', axis=1)
    train_set = train_set.join(one_hot).astype(float)

    one_hot_columns = test_set['track'].unique()
    one_hot = pd.get_dummies(test_set['track'])
    test_set = test_set.drop('track', axis=1)
    test_set = test_set.join(one_hot).astype(float)

    X_train = train_set.drop(columns=one_hot_columns).values.reshape(-1, 30)
    y_train = train_set[one_hot_columns].values.reshape(-1, num_classes)
    X_test = test_set.drop(columns=one_hot_columns).values.reshape(-1, 30)
    y_test = test_set[one_hot_columns].values.reshape(-1, num_classes)

    model = create_model(num_classes, activation=config['activation'], hidden_layers=config['hidden_layers'], dropout_rate=config['dropout_rate'])

    # Train the model and collect performance data
    model= train(model=model, X_train=X_train, y_train=y_train, num_epochs=config['epochs'], batch_size=config['batch_size'], learning_rate=config['learning_rate'])
    # Evaluate the model and collect performance data
    accuracy, f1_macro, f1_micro, mcc = evaluate(model=model, X_test=X_test, y_test=y_test)


    return accuracy, f1_macro, f1_micro, mcc, num_classes

## 5-Fold-Cross Validation

In [6]:
df_data_spike_1_split = pd.DataFrame()
df_data_spike_full_split = pd.DataFrame()

data_spike_exec_1_split_dict = dict()
data_spike_exec_full_split_dict = dict()


best_params_list_getting = []

def custom_trial_dirname(trial):
    return f"trial_{trial.trial_id}"

In [7]:
def train_and_evaluate(config, splits, splits_name):
    
    global data_spike_exec_1_split_dict
    global data_spike_exec_full_split_dict
    global df_data_spike_1_split
    global results_dir
    
    min_accuracy = 1
    max_accuracy = 0
    avg_accuracy = 0
    min_f1_macro = 1
    max_f1_macro = 0
    avg_f1_macro = 0
    min_f1_micro = 1
    max_f1_micro = 0
    avg_f1_micro = 0
    min_mcc = 1
    max_mcc = -1
    avg_mcc = 0

    num_classes = 0
    
    accuracies = []
    f1_macro_scores = []
    f1_micro_scores = []
    mcc_scores = []

    session_id_for_df = session.get_trial_id()
    
    for i in range(5):
        accuracy, f1_macro, f1_micro, mcc, num_classes = evaluate_model_on_dataset_one_split(config, splits, i)
        accuracies.append(accuracy)
        f1_macro_scores.append(f1_macro)
        f1_micro_scores.append(f1_micro)
        mcc_scores.append(mcc)
        
        avg_accuracy += accuracy
        avg_f1_macro += f1_macro
        avg_f1_micro += f1_micro
        min_accuracy = min(min_accuracy, accuracy)
        max_accuracy = max(max_accuracy, accuracy)
        min_f1_macro = min(min_f1_macro, f1_macro)
        max_f1_macro = max(max_f1_macro, f1_macro)
        min_f1_micro = min(min_f1_micro, f1_micro)
        max_f1_micro = max(max_f1_micro, f1_micro)
        avg_mcc += mcc
        min_mcc = min(min_mcc, mcc)
        max_mcc = max(max_mcc, mcc)


        
        data_spike_exec_1_split_dict[splits_name + "_" + str(i+1) + "_" + session_id_for_df] = {
            "number of classes": num_classes,
            "accuracy": accuracy,
            "macro f1": f1_macro,
            "micro_f1": f1_micro,
            "mcc": mcc,
            "config": str(config)
        }
        
       
    temp_df = pd.DataFrame.from_dict(data_spike_exec_1_split_dict, orient='index')
    df_data_spike_1_split = pd.concat([df_data_spike_1_split, temp_df], axis=0)

        # df_data_spike_1_split = pd.concat([df_data_spike_1_split, pd.DataFrame.from_dict(data_spike_exec_1_split_dict)], axis=1)

        
    avg_accuracy /= 5
    avg_f1_macro /= 5
    avg_f1_micro /= 5
    avg_mcc /= 5
    
    data_spike_exec_full_split_dict[splits_name + "_" + session_id_for_df] = {
        "number of classes": num_classes,
        "min_accuracy": min_accuracy,
        "max_accuracy": max_accuracy,
        "mean_accuracy": np.mean(accuracies),
        "min_f1_macro": min_f1_macro,
        "max_f1_macro": max_f1_macro,
        "mean_f1_macro": np.mean(f1_macro_scores),
        "min_f1_micro": min_f1_micro,
        "max_f1_micro": max_f1_micro,
        "mean_f1_micro": np.mean(f1_micro_scores),
        "min_mcc": min_mcc,
        "max_mcc": max_mcc,
        "mean_mcc": np.mean(mcc_scores),
        "std_accuracy": np.std(accuracies),
        "std_f1_macro": np.std(f1_macro_scores),
        "std_f1_micro": np.std(f1_micro_scores),
        "std_mcc": np.std(mcc_scores),
        "config": str(config)
    }

    # print(f"Mean accuracy of the model {splits_name + '_' + session_id_for_df}: {avg_accuracy}")

    df_data_spike_full_split = pd.DataFrame.from_dict(data_spike_exec_full_split_dict, orient='index')

    # Load existing data from file if it exists and append new data
    full_split_path = os.path.join(helper.results_dir, f'output_full_{splits_name}.pkl')
    if os.path.exists(full_split_path):
        df_existing_full = pd.read_pickle(full_split_path)
        df_data_spike_full_split = pd.concat([df_existing_full, df_data_spike_full_split], axis=0)
    else:
        print(f"No existing full split data found at {full_split_path}, creating new file.")
    
    # Save the updated full split data to file
    df_data_spike_full_split.to_pickle(full_split_path)

    # Save the 1 split data using the full path
    split_path = os.path.join(helper.results_dir, f'output_{splits_name}.pkl')
    if os.path.exists(split_path):
        df_existing_1_spike = pd.read_pickle(split_path)
        df_data_spike_1_split = pd.concat([df_existing_1_spike, df_data_spike_1_split], axis=0)
    else:
        print(f"No existing 1 split data found at {split_path}, creating new file")

    # Save the updated 1 split data to file
    df_data_spike_1_split.to_pickle(split_path)

    time.sleep(5)
    
    session.report({
        "min_accuracy": min_accuracy,
        "max_accuracy": max_accuracy,
        "mean_accuracy": np.mean(accuracies),
        "min_f1_macro": min_f1_macro,
        "max_f1_macro": max_f1_macro,
        "mean_f1_macro": np.mean(f1_macro_scores),
        "min_f1_micro": min_f1_micro,
        "max_f1_micro": max_f1_micro,
        "mean_f1_micro": np.mean(f1_micro_scores)
    })

In [8]:
def generate_hidden_layers_config(min_layers=2, max_layers=5, min_nodes=30, max_nodes=90, step=5):
    possible_layers = []
    for num_layers in range(min_layers, max_layers + 1):
        layer_configurations = list(itertools.product(range(min_nodes, max_nodes + 1, step), repeat=num_layers))
        possible_layers.extend(layer_configurations)
    return possible_layers

def five_fold_cross_validation(splits, splits_name):

    global best_params_list_getting

    hidden_layers_options = generate_hidden_layers_config()

    config = {
        "activation": tune.choice(["relu", "tanh", "sigmoid"]),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128]),
        "hidden_layers": tune.choice(hidden_layers_options),
        "epochs": tune.choice([10, 20, 30, 40, 50]),
        "dropout_rate": tune.uniform(0.2, 0.5)
    }
    
    scheduler = ASHAScheduler(
        metric="mean_accuracy",
        mode="max",
        max_t=10,
        grace_period=1,
        reduction_factor=2
    )
    
    search_alg = OptunaSearch(metric="mean_accuracy", mode="max")
    
    analysis = tune.run(
        tune.with_parameters(train_and_evaluate, splits=splits, splits_name=splits_name),
        resources_per_trial={"cpu": 10, "gpu": 0, "accelerator_type:RTX": 0},
        config=config,
        scheduler=scheduler,
        search_alg=search_alg,
        num_samples=5,
        verbose=1,
        storage_path=helper.ray_results_dir,
        trial_dirname_creator=custom_trial_dirname,
        callbacks=[WandbLoggerCallback(
            project="Classic FFN PyTorch - Test WandB",
            api_key="06649006f62a4b3df7b4f3a86fa05340fb7a736f",
            log_config=True
        )]
    )

    best_config_data_ray_tune = analysis.get_best_config(metric="mean_accuracy", mode="max")
    print("Best hyperparameters found were: ", best_config_data_ray_tune)
    best_params_list_getting.append(best_config_data_ray_tune)
    
    return analysis

In [9]:
analysis = five_fold_cross_validation(helper.a2_splits, "A2")

(train_and_evaluate pid=52941) wandb: Currently logged in as: bhanu-prasanna2001. Use `wandb login --relogin` to force relogin
(train_and_evaluate pid=52941) wandb: Tracking run with wandb version 0.17.6
(train_and_evaluate pid=52941) wandb: Run data is saved locally in /private/tmp/ray/session_2024-08-15_10-43-09_737131_52755/artifacts/2024-08-15_10-43-36/train_and_evaluate_2024-08-15_10-43-36/working_dirs/trial_768fd1ba/wandb/run-20240815_104344-a4w0vc5w
(train_and_evaluate pid=52941) wandb: Run `wandb offline` to turn off syncing.
(train_and_evaluate pid=52941) wandb: Syncing run trial_768fd1ba
(train_and_evaluate pid=52941) wandb: ⭐️ View project at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging
(train_and_evaluate pid=52941) wandb: 🚀 View run at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging/runs/a4w0vc5w
wandb:                                                                                
(_Wandb

(train_and_evaluate pid=52941) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A2.pkl, creating new file.
(train_and_evaluate pid=52941) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A2.pkl, creating new file


2024-08-15 10:44:30,592	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.712 s, which may be a performance bottleneck.
2024-08-15 10:44:30,593	WARNING util.py:201 -- The `process_trial_result` operation took 0.713 s, which may be a performance bottleneck.
2024-08-15 10:44:30,593	WARNING util.py:201 -- Processing trial results took 0.713 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-08-15 10:44:30,594	WARNING util.py:201 -- The `process_trial_result` operation took 0.714 s, which may be a performance bottleneck.
2024-08-15 10:44:31,287	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 30, 45, 50, 30)}
wandb:                                                                                
(_WandbLoggingActor pid=52951) wandb: 
(_WandbLoggingActor pid=52951) wandb: Run history:
(_WandbLoggingActor pid=52951) wandb:        config/ba

Best hyperparameters found were:  {'activation': 'sigmoid', 'learning_rate': 0.00320027036491653, 'batch_size': 32, 'hidden_layers': (50, 55, 60, 85, 90), 'epochs': 20, 'dropout_rate': 0.40970736826155685}


In [10]:
analysis = five_fold_cross_validation(helper.a3_splits, "A3")

(train_and_evaluate pid=55020) wandb: Currently logged in as: bhanu-prasanna2001. Use `wandb login --relogin` to force relogin
(train_and_evaluate pid=55020) wandb: Tracking run with wandb version 0.17.6
(train_and_evaluate pid=55020) wandb: Run data is saved locally in /private/tmp/ray/session_2024-08-15_10-43-09_737131_52755/artifacts/2024-08-15_10-49-01/train_and_evaluate_2024-08-15_10-49-01/working_dirs/trial_065f6212/wandb/run-20240815_104908-8uq6u5y3
(train_and_evaluate pid=55020) wandb: Run `wandb offline` to turn off syncing.
(train_and_evaluate pid=55020) wandb: Syncing run trial_065f6212
(train_and_evaluate pid=55020) wandb: ⭐️ View project at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging
(train_and_evaluate pid=55020) wandb: 🚀 View run at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging/runs/8uq6u5y3
wandb:                                                                                
(_Wandb

(train_and_evaluate pid=55020) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A3.pkl, creating new file.
(train_and_evaluate pid=55020) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A3.pkl, creating new file


2024-08-15 10:49:55,049	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.585 s, which may be a performance bottleneck.
2024-08-15 10:49:55,050	WARNING util.py:201 -- The `process_trial_result` operation took 0.586 s, which may be a performance bottleneck.
2024-08-15 10:49:55,050	WARNING util.py:201 -- Processing trial results took 0.586 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-08-15 10:49:55,050	WARNING util.py:201 -- The `process_trial_result` operation took 0.586 s, which may be a performance bottleneck.
2024-08-15 10:49:55,854	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (75, 60, 90, 85, 55)}
wandb:                                                                                
(train_and_evaluate pid=55020) wandb: - 0.002 MB of 0.002 MB uploaded
(_WandbLoggingActor pid=55048) wandb: 
(_WandbLoggingActor pid=55048) wand

Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.0005835127081408091, 'batch_size': 32, 'hidden_layers': (40, 40, 80, 75, 85), 'epochs': 50, 'dropout_rate': 0.2583779379085752}


(_WandbLoggingActor pid=56387) wandb: 
(_WandbLoggingActor pid=56387) wandb: Run history:
(_WandbLoggingActor pid=56387) wandb:        config/batch_size ▁
(_WandbLoggingActor pid=56387) wandb:      config/dropout_rate ▁
(_WandbLoggingActor pid=56387) wandb:            config/epochs ▁
(_WandbLoggingActor pid=56387) wandb:     config/learning_rate ▁
(_WandbLoggingActor pid=56387) wandb: iterations_since_restore ▁
(_WandbLoggingActor pid=56387) wandb:             max_accuracy ▁
(_WandbLoggingActor pid=56387) wandb:             max_f1_macro ▁
(_WandbLoggingActor pid=56387) wandb:             max_f1_micro ▁
(_WandbLoggingActor pid=56387) wandb:            mean_accuracy ▁
(_WandbLoggingActor pid=56387) wandb:            mean_f1_macro ▁
(_WandbLoggingActor pid=56387) wandb:            mean_f1_micro ▁
(_WandbLoggingActor pid=56387) wandb:             min_accuracy ▁
(_WandbLoggingActor pid=56387) wandb:             min_f1_macro ▁
(_WandbLoggingActor pid=56387) wandb:             min_f1_micro ▁


In [11]:
analysis = five_fold_cross_validation(helper.a4_splits, "A4")

(train_and_evaluate pid=56853) wandb: Currently logged in as: bhanu-prasanna2001. Use `wandb login --relogin` to force relogin
(train_and_evaluate pid=56853) wandb: Tracking run with wandb version 0.17.6
(train_and_evaluate pid=56853) wandb: Run data is saved locally in /private/tmp/ray/session_2024-08-15_10-43-09_737131_52755/artifacts/2024-08-15_10-53-40/train_and_evaluate_2024-08-15_10-53-40/working_dirs/trial_e701240e/wandb/run-20240815_105346-fx7pby2i
(train_and_evaluate pid=56853) wandb: Run `wandb offline` to turn off syncing.
(train_and_evaluate pid=56853) wandb: Syncing run trial_e701240e
(train_and_evaluate pid=56853) wandb: ⭐️ View project at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging
(train_and_evaluate pid=56853) wandb: 🚀 View run at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging/runs/fx7pby2i
wandb:                                                                                
(_Wandb

(train_and_evaluate pid=56853) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A4.pkl, creating new file.
(train_and_evaluate pid=56853) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A4.pkl, creating new file


2024-08-15 10:54:41,373	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.615 s, which may be a performance bottleneck.
2024-08-15 10:54:41,374	WARNING util.py:201 -- The `process_trial_result` operation took 0.616 s, which may be a performance bottleneck.
2024-08-15 10:54:41,374	WARNING util.py:201 -- Processing trial results took 0.617 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-08-15 10:54:41,374	WARNING util.py:201 -- The `process_trial_result` operation took 0.617 s, which may be a performance bottleneck.
2024-08-15 10:54:42,069	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (55, 85, 40, 80)}
wandb:                                                                                
(_WandbLoggingActor pid=56861) wandb: 
(_WandbLoggingActor pid=56861) wandb: Run history:
(_WandbLoggingActor pid=56861) wandb:        config/batch_

Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0008145249780491403, 'batch_size': 64, 'hidden_layers': (35, 55, 60, 65, 40), 'epochs': 40, 'dropout_rate': 0.3476360245730923}


(_WandbLoggingActor pid=58860) wandb: 
(_WandbLoggingActor pid=58860) wandb: Run history:
(_WandbLoggingActor pid=58860) wandb:        config/batch_size ▁
(_WandbLoggingActor pid=58860) wandb:      config/dropout_rate ▁
(_WandbLoggingActor pid=58860) wandb:            config/epochs ▁
(_WandbLoggingActor pid=58860) wandb:     config/learning_rate ▁
(_WandbLoggingActor pid=58860) wandb: iterations_since_restore ▁
(_WandbLoggingActor pid=58860) wandb:             max_accuracy ▁
(_WandbLoggingActor pid=58860) wandb:             max_f1_macro ▁
(_WandbLoggingActor pid=58860) wandb:             max_f1_micro ▁
(_WandbLoggingActor pid=58860) wandb:            mean_accuracy ▁
(_WandbLoggingActor pid=58860) wandb:            mean_f1_macro ▁
(_WandbLoggingActor pid=58860) wandb:            mean_f1_micro ▁
(_WandbLoggingActor pid=58860) wandb:             min_accuracy ▁
(_WandbLoggingActor pid=58860) wandb:             min_f1_macro ▁
(_WandbLoggingActor pid=58860) wandb:             min_f1_micro ▁


In [12]:
analysis = five_fold_cross_validation(helper.a12_splits, "A12")

(train_and_evaluate pid=59324) wandb: Currently logged in as: bhanu-prasanna2001. Use `wandb login --relogin` to force relogin
(train_and_evaluate pid=59324) wandb: Tracking run with wandb version 0.17.6
(train_and_evaluate pid=59324) wandb: Run data is saved locally in /private/tmp/ray/session_2024-08-15_10-43-09_737131_52755/artifacts/2024-08-15_10-59-31/train_and_evaluate_2024-08-15_10-59-31/working_dirs/trial_c944a56d/wandb/run-20240815_105937-57jldx1q
(train_and_evaluate pid=59324) wandb: Run `wandb offline` to turn off syncing.
(train_and_evaluate pid=59324) wandb: Syncing run trial_c944a56d
(train_and_evaluate pid=59324) wandb: ⭐️ View project at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging
(train_and_evaluate pid=59324) wandb: 🚀 View run at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging/runs/57jldx1q
wandb:                                                                                
(_Wandb

(train_and_evaluate pid=59324) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A12.pkl, creating new file.
(train_and_evaluate pid=59324) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A12.pkl, creating new file


2024-08-15 11:00:32,885	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.669 s, which may be a performance bottleneck.
2024-08-15 11:00:32,886	WARNING util.py:201 -- The `process_trial_result` operation took 0.671 s, which may be a performance bottleneck.
2024-08-15 11:00:32,886	WARNING util.py:201 -- Processing trial results took 0.672 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-08-15 11:00:32,887	WARNING util.py:201 -- The `process_trial_result` operation took 0.672 s, which may be a performance bottleneck.
2024-08-15 11:00:33,577	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (60, 55, 75, 90)}
wandb:                                                                                
(_WandbLoggingActor pid=59344) wandb: 
(_WandbLoggingActor pid=59344) wandb: Run history:
(_WandbLoggingActor pid=59344) wandb:        config/batch_

Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0001288241705962323, 'batch_size': 32, 'hidden_layers': (55, 75, 70, 50, 60), 'epochs': 50, 'dropout_rate': 0.2516006506162699}


In [13]:
analysis = five_fold_cross_validation(helper.a21_splits, "A21")

(train_and_evaluate pid=62164) wandb: Currently logged in as: bhanu-prasanna2001. Use `wandb login --relogin` to force relogin
(train_and_evaluate pid=62164) wandb: Tracking run with wandb version 0.17.6
(train_and_evaluate pid=62164) wandb: Run data is saved locally in /private/tmp/ray/session_2024-08-15_10-43-09_737131_52755/artifacts/2024-08-15_11-06-17/train_and_evaluate_2024-08-15_11-06-17/working_dirs/trial_5867577c/wandb/run-20240815_110623-g21ve12z
(train_and_evaluate pid=62164) wandb: Run `wandb offline` to turn off syncing.
(train_and_evaluate pid=62164) wandb: Syncing run trial_5867577c
(train_and_evaluate pid=62164) wandb: ⭐️ View project at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging
(train_and_evaluate pid=62164) wandb: 🚀 View run at https://wandb.ai/bhanu-prasanna2001/Classic%20FFN%20PyTorch%20-%20Train%20Loss%20Logging/runs/g21ve12z
(train_and_evaluate pid=62164) wandb: - 0.002 MB of 0.015 MB uploaded
(_WandbLoggingActor pid=

(train_and_evaluate pid=62164) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A21.pkl, creating new file.
(train_and_evaluate pid=62164) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A21.pkl, creating new file


2024-08-15 11:07:14,689	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 0.624 s, which may be a performance bottleneck.
2024-08-15 11:07:14,689	WARNING util.py:201 -- The `process_trial_result` operation took 0.625 s, which may be a performance bottleneck.
2024-08-15 11:07:14,690	WARNING util.py:201 -- Processing trial results took 0.625 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-08-15 11:07:14,690	WARNING util.py:201 -- The `process_trial_result` operation took 0.626 s, which may be a performance bottleneck.
2024-08-15 11:07:15,382	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 60, 90, 80, 35)}
wandb:                                                                                
(_WandbLoggingActor pid=62172) wandb: 
(_WandbLoggingActor pid=62172) wandb: Run history:
(_WandbLoggingActor pid=62172) wandb:        config/ba

Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.00299684677031004, 'batch_size': 64, 'hidden_layers': (55, 30, 75, 45, 50), 'epochs': 40, 'dropout_rate': 0.22849097764163756}


In [14]:
df_dict_A2 = pd.read_pickle("results/output_A2.pkl")
df_dict_A3 = pd.read_pickle("results/output_A3.pkl")
df_dict_A4 = pd.read_pickle("results/output_A4.pkl")
df_dict_A12 = pd.read_pickle("results/output_A12.pkl")
df_dict_A21 = pd.read_pickle("results/output_A21.pkl")

df_dict = pd.concat([df_dict_A2, df_dict_A3, df_dict_A4, df_dict_A12, df_dict_A21])

df_dict = df_dict.loc[:,~df_dict.columns.duplicated()].copy()
df_dict = df_dict.reset_index().rename(columns={'index': 'name'})
df_dict = df_dict.drop_duplicates()

df_dict.to_csv('outputs/Classic_FFN_one_split_metrics_system.csv')

In [15]:
df_dict_full_A2 = pd.read_pickle("results/output_full_A2.pkl")
df_dict_full_A3 = pd.read_pickle("results/output_full_A3.pkl")
df_dict_full_A4 = pd.read_pickle("results/output_full_A4.pkl")
df_dict_full_A12 = pd.read_pickle("results/output_full_A12.pkl")
df_dict_full_A21 = pd.read_pickle("results/output_full_A21.pkl")

df_full_dict = pd.concat([df_dict_full_A2, df_dict_full_A3, df_dict_full_A4, df_dict_full_A12, df_dict_full_A21])

df_full_dict = df_full_dict.loc[:, ~df_full_dict.columns.duplicated()].copy()
df_full_dict = df_full_dict.reset_index().rename(columns={'index': 'name'})
df_full_dict = df_full_dict.drop_duplicates()

df_full_dict.to_csv("outputs/Classic_FFN_full_split_metrics_system.csv")

In [16]:
df_dict_full_A2.to_csv("outputs/Classic_FFN_full_split_metrics_A2.csv")
df_dict_full_A3.to_csv("outputs/Classic_FFN_full_split_metrics_A3.csv")
df_dict_full_A4.to_csv("outputs/Classic_FFN_full_split_metrics_A4.csv")
df_dict_full_A12.to_csv("outputs/Classic_FFN_full_split_metrics_A12.csv")
df_dict_full_A21.to_csv("outputs/Classic_FFN_full_split_metrics_A21.csv")

In [17]:
# print the configs and results for the hyperparemters wiht the highest mean accuracy
# read df_full_dict and print columns with configs in best_params_list_getting
df_full_dict = pd.read_csv("outputs/Classic_FFN_full_split_metrics_system.csv")
if 'Unnamed: 0' in df_full_dict.columns:
    df_full_dict.drop(columns=['Unnamed: 0'], inplace=True)

config_acc_dict = {}
for good_param in best_params_list_getting:
    # get the row witht the highest mean mean accuracy
    mean_accuracy = df_full_dict.loc[df_full_dict['config'] == str(good_param)]['mean_accuracy'].mean()
    config_acc_dict[str(good_param)] = mean_accuracy
    print(f"Mean accuracy for config {good_param} is {mean_accuracy}")

# get the best config
best_config = max(config_acc_dict, key=config_acc_dict.get)
print(f"\n\nThe best config is {best_config} with a mean accuracy of {config_acc_dict[best_config]}")

Mean accuracy for config {'activation': 'sigmoid', 'learning_rate': 0.00320027036491653, 'batch_size': 32, 'hidden_layers': (50, 55, 60, 85, 90), 'epochs': 20, 'dropout_rate': 0.40970736826155685} is 0.8657667012807199
Mean accuracy for config {'activation': 'relu', 'learning_rate': 0.0005835127081408091, 'batch_size': 32, 'hidden_layers': (40, 40, 80, 75, 85), 'epochs': 50, 'dropout_rate': 0.2583779379085752} is 0.8728778467908903
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.0008145249780491403, 'batch_size': 64, 'hidden_layers': (35, 55, 60, 65, 40), 'epochs': 40, 'dropout_rate': 0.3476360245730923} is 0.6683845468283798
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.0001288241705962323, 'batch_size': 32, 'hidden_layers': (55, 75, 70, 50, 60), 'epochs': 50, 'dropout_rate': 0.2516006506162699} is 0.2893199983933807
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.00299684677031004, 'batch_size': 64, 'hidden_layers': (55, 30, 

In [18]:
best_performers = df_full_dict.loc[df_full_dict.groupby(df_full_dict['name'].str.split('_').str[0])['mean_accuracy'].idxmax()]
best_performers = best_performers.sort_index().reset_index(drop=True)

best_performers

,name,number of classes,min_accuracy,max_accuracy,mean_accuracy,min_f1_macro,max_f1_macro,mean_f1_macro,min_f1_micro,max_f1_micro,mean_f1_micro,min_mcc,max_mcc,mean_mcc,std_accuracy,std_f1_macro,std_f1_micro,std_mcc,config
0,A2_e835aaec,2,0.814815,0.897196,0.865767,0.814815,0.896618,0.865600,0.814815,0.897196,0.865767,0.629630,0.806152,0.734311,0.032146,0.031998,0.032146,0.066947,"{'activation': 'sigmoid', 'learning_rate': 0.0..."
1,A3_9f8fa340,2,0.811594,0.927536,0.872878,0.811594,0.926984,0.872380,0.811594,0.927536,0.872878,0.623529,0.863900,0.751752,0.038055,0.038057,0.038055,0.077365,"{'activation': 'relu', 'learning_rate': 0.0005..."
2,A4_e204b1d6,3,0.624365,0.712468,0.668385,0.615275,0.708924,0.661755,0.624365,0.712468,0.668385,0.464322,0.569499,0.509967,0.027969,0.030233,0.027969,0.034400,"{'activation': 'tanh', 'learning_rate': 0.0008..."
3,A12_93943072,6,0.266839,0.307494,0.289320,0.247697,0.284539,0.269003,0.266839,0.307494,0.289320,0.122156,0.171728,0.149201,0.017598,0.015025,0.017598,0.021320,"{'activation': 'tanh', 'learning_rate': 0.0001..."
4,A21_c243c28d,2,0.647619,0.733333,0.693890,0.595462,0.731360,0.651514,0.647619,0.733333,0.693890,0.276208,0.471365,0.345755,0.035011,0.051874,0.035011,0.078715,"{'activation': 'tanh', 'learning_rate': 0.0029..."
